chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://cse.buffalo.edu/faculty/miller/Courses/CSE633/Aashna-Mahajan-Spring-2020.pdf

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasetRGB.csv')
image_array = df[['Red', 'Green', 'Blue']].values

In [3]:
from sklearn.cluster import KMeans
import numpy as np

In [5]:
#Reshape the image
shape = image_array.shape
reshaped_image_array = image_array.reshape(-1, 3)

In [6]:
#Apply k-means clustering
kmeans = KMeans(n_clusters=5)
kmeans.fit(reshaped_image_array)
labels = kmeans.predict(reshaped_image_array)
compressed_colors = kmeans.cluster_centers_.astype(int)

c:\Users\Advay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [7]:
#Replace pixel values
compressed_image_array = compressed_colors[labels]
compressed_image_array = compressed_image_array.reshape(shape)